In [166]:
#Import Dependencies
import pandas as pd
from datetime import date
from pathlib import Path

In [167]:
#Set Paths for Data CSV's
clinp=Path("Resources/WHO_NREVSS_Clinical_Labs.csv")
pubp=Path("Resources/WHO_NREVSS_Public_Health_Labs.csv")
comp=Path("Resources/WHO_NREVSS_Combined_prior_to_2015_16.csv")

In [168]:
#Read Data from the CSV's
clinical = pd.read_csv(clinp,header=1)
public = pd.read_csv(pubp, header=1)
combo2015= pd.read_csv(comp, header=1)

clinical.head()

,REGION TYPE,REGION,YEAR,WEEK,TOTAL SPECIMENS,TOTAL A,TOTAL B,PERCENT POSITIVE,PERCENT A,PERCENT B
0,States,Alabama,2015,40,167,2,3,2.99,1.2,1.8
1,States,Alaska,2015,40,X,X,X,X,X,X
2,States,Arizona,2015,40,55,0,0,0,0,0
3,States,Arkansas,2015,40,26,0,1,3.85,0,3.85
4,States,California,2015,40,683,2,0,0.29,0.29,0


In [169]:
#Correct the 2014 week 53 which must have been a recording error, as 2014 only had 52 weeks per the ISO calendar
combo2015.loc[(combo2015["WEEK"]==53),"WEEK"]=52

#Isolate the data taken during flu season for weekly data (October-May)
clinical["Season"]=clinical[["YEAR","WEEK"]].apply(lambda row: date.fromisocalendar(int(row['YEAR']), int(row['WEEK']),1).month,axis=1)
clinical=clinical.drop(clinical[(clinical.Season<10)&(clinical.Season>5)].index)

combo2015["Season"]=combo2015[["YEAR","WEEK"]].apply(lambda row: date.fromisocalendar(int(row['YEAR']), int(row['WEEK']),1).month,axis=1)
combo2015=combo2015.drop(combo2015[(combo2015.Season<10)&(combo2015.Season>5)].index)

In [170]:
#Drop Null (X only) entries
clinical=clinical[clinical["TOTAL SPECIMENS"]!="X"]
public=public[public["TOTAL SPECIMENS"]!="X"]
combo2015=combo2015[combo2015["TOTAL SPECIMENS"]!="X"]


#Replace isolated X's with 0's
for coln in [5,6]:
    col=clinical.columns[coln]
    clinical.loc[clinical[col]=="X",col]=0

for coln in range(4,12):
    col=public.columns[coln]
    public.loc[public[col]=="X",col]=0

for coln in range(6,14):
    col=combo2015.columns[coln]
    combo2015.loc[combo2015[col]=="X",col]=0


In [171]:
#Convert date data into "Season year1-year2" format present in the Public dataset
clinical.loc[(clinical["Season"]<10),"YEAR"]=clinical["YEAR"].apply(lambda x: x-1)
clinical["YEAR_2"]=clinical["YEAR"].apply(lambda x: str(x+1)[2:4])
clinical["Season"]=clinical[["YEAR","YEAR_2"]].apply(lambda row: "Season "+str(row["YEAR"])+"-"+row["YEAR_2"],axis=1)

combo2015.loc[(combo2015["Season"]<10),"YEAR"]=combo2015["YEAR"].apply(lambda x: x-1)
combo2015["YEAR_2"]=combo2015["YEAR"].apply(lambda x: str(x+1)[2:4])
combo2015["Season"]=combo2015[["YEAR","YEAR_2"]].apply(lambda row: "Season "+str(row["YEAR"])+"-"+row["YEAR_2"],axis=1)

#Convert Numerical Data into integers
clinical=clinical.astype({"TOTAL SPECIMENS":"int32","TOTAL A":"int32","TOTAL B":"int32"})
combo2015=combo2015.astype({"TOTAL SPECIMENS":"int32","A (2009 H1N1)":"int32","A (H1)":"int32","A (H3)":"int32",
                            "A (Subtyping not Performed)":"int32","A (Unable to Subtype)":"int32","B":"int32",
                            "H3N2v":"int32","A (H5)":"int32"})
public=public.astype({"TOTAL SPECIMENS":"int32","A (2009 H1N1)":"int32","A (H3)":"int32",
                            "A (Subtyping not Performed)":"int32","B":"int32","BVic":"int32","BYam":"int32",
                            "H3N2v":"int32","A (H5)":"int32"})

In [ ]:
#Drop Unnecessary Columns and Columns that can be recalculated after merge and cleaning. Rename columns more concisely
clinical.drop(["REGION TYPE","YEAR","WEEK","PERCENT POSITIVE","PERCENT A","PERCENT B","YEAR_2"],axis=1,inplace=True)
clinical.rename({"REGION":"State","TOTAL SPECIMENS":"Total Specimens","TOTAL A":"A (Subtype N/A)","TOTAL B":"B (Subtype N/A)"},
                axis=1,inplace=True)


public.drop("REGION TYPE",axis=1,inplace=True)
public.rename({"REGION":"State","SEASON_DESCRIPTION":"Season","TOTAL SPECIMENS":"Total Specimens",
               "A (Subtyping not Performed)":"A (Subtype N/A)","B":"B (Subtype N/A)","H3N2v":"A (H3N2v)"},axis=1,inplace=True)


combo2015.rename({"REGION":"State","TOTAL SPECIMENS":"Total Specimens","B":"B (Subtype N/A)","H3N2v":"A (H3N2v)"},
                axis=1,inplace=True)
#As far as all my searches online, the only A (H1) the routinely circulates in humans is the H1N1, so combining the columns
combo2015["A (2009 H1N1)"]=combo2015["A (2009 H1N1)"]+combo2015["A (H1)"]

#For our purposes, the reason the subtyping was not performed is irrelevant, so combining columns as a general Subtype N/A
combo2015["A (Subtype N/A)"]=combo2015["A (Subtyping not Performed)"]+combo2015["A (Unable to Subtype)"]
combo2015.drop(["REGION TYPE","YEAR","WEEK","PERCENT POSITIVE","YEAR_2","A (H1)","A (Subtyping not Performed)",
                "A (Unable to Subtype)"],axis=1,inplace=True)


#Merge all three cleaned dataframes into a master list
fludata=pd.concat([clinical,public,combo2015],ignore_index=True)
fludata.fillna(value=0,inplace=True)
fludata=fludata.astype({"A (2009 H1N1)":"int32","A (H3)":"int32","BVic":"int32","BYam":"int32","A (H3N2v)":"int32","A (H5)":"int32"})

#Create A and B totals
fludata["Total A"]=fludata["A (2009 H1N1)"]+fludata["A (H3)"]+fludata["A (H5)"]+fludata["A (Subtype N/A)"]+fludata["A (H3N2v)"]
fludata["Total B"]=fludata["B (Subtype N/A)"]+fludata["BVic"]+fludata["BYam"]
fludata.head()

,State,Total Specimens,A (Subtype N/A),B (Subtype N/A),Season,A (2009 H1N1),A (H3),BVic,BYam,A (H3N2v),A (H5),Total A,Total B
0,Alabama,167,2,1,Season 2015-16,0,0,0,0,0,0,2,1
1,Arizona,71,0,0,Season 2015-16,0,0,0,0,0,0,0,0
2,Arkansas,36,1,0,Season 2015-16,0,0,0,0,0,0,1,0
3,California,661,3,2,Season 2015-16,0,0,0,0,0,0,3,2
4,Colorado,281,2,0,Season 2015-16,0,0,0,0,0,0,2,0


In [ ]:
#Groupby Season and Region, summing the weekly data into season data
fludata=fludata[["State","Season","Total Specimens","Total A","Total B","A (Subtype N/A)","A (2009 H1N1)","A (H3)","A (H3N2v)","A (H5)",
                 "B (Subtype N/A)","BVic","BYam"]].groupby(["State","Season"]).sum(numeric_only=1)
#Export Master Dataframe to be used for further visualization
flumaster=fludata
flumaster.to_csv('Statedata/1FluMaster.csv',header=True)

#Export Flu Data by state for storage on database
fludata=fludata.reset_index()
st=fludata["State"].unique()
for s in range(0,53):
    state=st[s]
    state2=state.replace(" ","")
    exec(state2+"df=fludata[fludata['State']==state].drop(columns=['State'])")
    exec(state2+"df.reset_index(drop=True,inplace=True)")
    exec(state2+"df.to_csv('Statedata/'+state2+'.csv',index=False,header=True)")